# Model to STM32 Conversion using ST Edge AI

This notebook demonstrates how to convert machine learning models to STM32-compatible code using ST Edge AI tools.

We'll convert:
1. CenterFace TFLite model for face detection
2. MobileFaceNet ONNX model for face recognition

## Setup and Dependencies

In [5]:
import os
import subprocess
import json
from pathlib import Path

# Use STM32CubeMX version that works (v2.1.0 instead of v2.2.0)
STEDGEAI_PATH = '/home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai'

# Test if stedgeai is accessible at the STM32CubeMX path
if os.path.exists(STEDGEAI_PATH):
    print(f"✅ stedgeai found at: {STEDGEAI_PATH}")
    
    # Test if it's executable and check for AVX requirements
    try:
        result = subprocess.run([STEDGEAI_PATH, '--help'], capture_output=True, text=True, timeout=5)
        print("✅ stedgeai is executable and responding")
        # Check version
        if 'v2.1.0' in result.stdout:
            print("✅ Using STM32CubeMX compatible version (v2.1.0)")
        else:
            print("ℹ️  Version info not found in help output")
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
        print(f"⚠️  stedgeai found but may have issues: {e}")
    except Exception as e:
        if "AVX" in str(e) or "SIGILL" in str(e):
            print("❌ AVX instruction set not available - this is likely the issue")
            print("Solutions:")
            print("1. Enable AVX in VM settings (see instructions)")
            print("2. Use a different machine with AVX support")
            print("3. Try running stedgeai with compatibility flags")
else:
    print(f"❌ stedgeai not found at: {STEDGEAI_PATH}")
    print("Please verify the installation path")

# Check CPU capabilities
print("\n=== CPU Capabilities Check ===")
try:
    result = subprocess.run(['lscpu'], capture_output=True, text=True)
    cpu_info = result.stdout
    if 'avx' in cpu_info.lower():
        print("✅ AVX support detected")
    else:
        print("❌ No AVX support detected")
        print("This VM needs AVX enabled in hypervisor settings")
except:
    print("Could not check CPU capabilities")

# Set up paths
models_dir = Path('./models')
output_dir = Path('./stm32_output')
output_dir.mkdir(exist_ok=True)

# Model paths
centerface_model = models_dir / 'centerface.tflite'
mobilefacenet_model = models_dir / 'mobilefacenet_real_faces_onnx.onnx'

print(f"CenterFace model exists: {centerface_model.exists()}")
print(f"MobileFaceNet model exists: {mobilefacenet_model.exists()}")

✅ stedgeai found at: /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai
✅ stedgeai is executable and responding
✅ Using STM32CubeMX compatible version (v2.1.0)

=== CPU Capabilities Check ===
✅ AVX support detected
CenterFace model exists: True
MobileFaceNet model exists: False


## Configuration for ST Edge AI

Create configuration files for optimized STM32 generation

## Memory Layout Strategy

The two models use different memory pool configurations to avoid conflicts in external flash:

**Memory Pool Assignments:**
- **CenterFace (model1.mpool)**: External flash starts at `0x71000000` 
- **MobileFaceNet (model2.mpool)**: External flash starts at `0x72000000`

**Flash Memory Layout:**
```
0x70000000 - 0x700FFFFF: Bootloader code (1MB)
0x70100000 - 0x701FFFFF: Application code (15MB)
0x71000000 - 0x71FFFFFF: CenterFace model data (16MB)
0x72000000 - 0x72FFFFFF: MobileFaceNet model data (16MB)
0x73000000 - 0x74FFFFFF: Available for other uses (32MB)
```

This layout ensures:
- No model data overwrites the bootloader or application
- Both models can coexist without conflicts
- Efficient memory utilization on STM32N6 with external flash

In [6]:
# Create neural art configuration for face detection (CenterFace)
# Uses model1.mpool with external flash at 0x71000000
face_detection_config = {
    "Globals": {},
    "Profiles": {
        "centerface": {
            "memory_pool": "./mempools/model1.mpool",
            "options": "-O3 --all-buffers-info --mvei --cache-maintenance --Oalt-sched --native-float --enable-virtual-mem-pools --Omax-ca-pipe 4 --Ocache-opt --Os --enable-epoch-controller"
        }
    }
}

# Create neural art configuration for face recognition (MobileFaceNet)
# Uses model2.mpool with external flash at 0x72000000 (different offset to avoid conflicts)
face_recognition_config = {
    "Globals": {},
    "Profiles": {
        "mobilefacenet": {
            "memory_pool": "./mempools/model2.mpool",
            "options": "-O3 --all-buffers-info --mvei --cache-maintenance --Oalt-sched --native-float --enable-virtual-mem-pools --Omax-ca-pipe 4 --Ocache-opt --Os --enable-epoch-controller"
        }
    }
}

# Save configurations
with open('face_detection_config.json', 'w') as f:
    json.dump(face_detection_config, f, indent=4)

with open('face_recognition_config.json', 'w') as f:
    json.dump(face_recognition_config, f, indent=4)

print("Configuration files created with proper memory pool assignments:")
print("- CenterFace: model1.mpool (external flash @ 0x71000000)")
print("- MobileFaceNet: model2.mpool (external flash @ 0x72000000)")
print("This prevents model data conflicts in external flash memory.")

Configuration files created with proper memory pool assignments:
- CenterFace: model1.mpool (external flash @ 0x71000000)
- MobileFaceNet: model2.mpool (external flash @ 0x72000000)
This prevents model data conflicts in external flash memory.


## Convert CenterFace TFLite Model

In [7]:
def run_stedgeai_conversion(model_path, output_name, target="stm32n6", input_data_type="uint8", neural_art_config="", profile_config=""):
    """Run ST Edge AI conversion for a model using STM32CubeMX configuration"""
    
    # Use the explicit path defined earlier
    if not os.path.exists(STEDGEAI_PATH):
        print(f"Error: stedgeai not found at {STEDGEAI_PATH}")
        return False
    
    print(f"Using stedgeai from: {STEDGEAI_PATH}")
    
    
    cmd = [
        STEDGEAI_PATH, "generate",
        "--name", str(output_name),
        "--model", str(model_path),
        "--target", target,
        "--st-neural-art", f"{profile_config}@{neural_art_config}",
        "--input-data-type", input_data_type,
        "--output", str(output_dir / output_name)
    ]
    
    print(f"Running command: {' '.join(cmd)}")
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print("STDOUT:", result.stdout)
        if result.stderr:
            print("STDERR:", result.stderr)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running stedgeai: {e}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        return False

# Convert CenterFace model
print("Converting CenterFace TFLite model...")
centerface_success = run_stedgeai_conversion(
    centerface_model, 
    'face_detection',
    target="stm32n6",
    input_data_type="float32",
    neural_art_config = "face_detection_config.json",
    profile_config = "centerface"
)

if centerface_success:
    print("✅ CenterFace model conversion completed successfully")
else:
    print("❌ CenterFace model conversion failed")

Converting CenterFace TFLite model...
Using stedgeai from: /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai
Running command: /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai generate --name face_detection --model models/centerface.tflite --target stm32n6 --st-neural-art centerface@face_detection_config.json --input-data-type float32 --output stm32_output/face_detection
STDOUT: ST Edge AI Core v2.1.0-20194 329b0e98d

PASS:  91%|█████████▏| 84/92 [00:06<00:01,  7.58it/s]
                                                     
 >>>> EXECUTING NEURAL ART COMPILER

PASS:  91%|█████████▏| 84/92 [00:06<00:01,  7.58it/s]
                                                     
   /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/atonn -i "/home/vboxuser/Desktop/Workshop/EdgeAI_Workshop/Exercise 2/stm32_output/face_detection/centerface_OE_3_2_0.onnx"

## Convert MobileFaceNet ONNX Model

In [4]:
# Convert MobileFaceNet model
print("Converting MobileFaceNet ONNX model...")
mobilefacenet_success = run_stedgeai_conversion(
    mobilefacenet_model, 
    'face_recognition',
    target="stm32n6",
    input_data_type="float32",
    neural_art_config = "face_recognition_config.json",
    profile_config = "mobilefacenet"
)

if mobilefacenet_success:
    print("✅ MobileFaceNet model conversion completed successfully")
else:
    print("❌ MobileFaceNet model conversion failed")

Converting MobileFaceNet ONNX model...
Using stedgeai from: /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai
Running command: /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai generate --name face_recognition --model models/mobilefacenet_real_faces_onnx.onnx --target stm32n6 --st-neural-art mobilefacenet@face_recognition_config.json --input-data-type float32 --output stm32_output/face_recognition
STDOUT: ST Edge AI Core v2.1.0-20194 329b0e98d

PASS:  88%|████████▊ | 81/92 [00:08<00:00, 24.14it/s]
                                                     

PASS:  88%|████████▊ | 81/92 [00:08<00:00, 24.14it/s]
                                                     

PASS:  88%|████████▊ | 81/92 [00:08<00:00, 24.14it/s]
                                                     

PASS:  88%|████████▊ | 81/92 [00:08<00:00, 24.14it/s]
                                                     

PASS:  88

## Post-processing and File Organization

In [12]:
def organize_output_files(model_name):
    """Organize the generated files for deployment"""
    
    model_output_dir = output_dir / model_name
    model_output_dir.mkdir(exist_ok=True)
    
    # Expected ST Edge AI output files
    st_ai_output = Path('st_ai_output')
    
    if st_ai_output.exists():
        # Copy main files
        files_to_copy = [
            'network.c',
            'network.h', 
            'network_ecblobs.h',
            'network_data.h'
        ]
        
        for file_name in files_to_copy:
            src_file = st_ai_output / file_name
            if src_file.exists():
                dst_file = model_output_dir / file_name
                subprocess.run(['cp', str(src_file), str(dst_file)])
                print(f"Copied {file_name} to {model_output_dir}")
            else:
                print(f"Warning: {file_name} not found in st_ai_output")
        
        # Handle binary data file if it exists
        binary_files = list(st_ai_output.glob('*.raw'))
        if binary_files:
            binary_file = binary_files[0]
            # Copy and convert to different formats
            bin_output = model_output_dir / f'{model_name}_data.bin'
            hex_output = model_output_dir / f'{model_name}_data.hex'
            
            subprocess.run(['cp', str(binary_file), str(bin_output)])
            
            # Convert to Intel HEX format
            try:
                subprocess.run([
                    'arm-none-eabi-objcopy', '-I', 'binary', str(bin_output),
                    '--change-addresses', '0x70380000', '-O', 'ihex', str(hex_output)
                ], check=True)
                print(f"Generated HEX file: {hex_output}")
            except subprocess.CalledProcessError:
                print("Warning: Could not generate HEX file (arm-none-eabi-objcopy not available)")
    
    return model_output_dir

# Organize output files
if centerface_success:
    centerface_dir = organize_output_files('centerface')
    print(f"CenterFace files organized in: {centerface_dir}")

if mobilefacenet_success:
    mobilefacenet_dir = organize_output_files('mobilefacenet')
    print(f"MobileFaceNet files organized in: {mobilefacenet_dir}")

CenterFace files organized in: stm32_output/centerface
MobileFaceNet files organized in: stm32_output/mobilefacenet


## Summary and Next Steps

In [13]:
print("\n" + "="*50)
print("CONVERSION SUMMARY")
print("="*50)

print(f"CenterFace TFLite → STM32: {'✅ Success' if centerface_success else '❌ Failed'}")
print(f"MobileFaceNet ONNX → STM32: {'✅ Success' if mobilefacenet_success else '❌ Failed'}")

print("\nGenerated files are organized in the ./stm32_output directory")
print("\nNext steps:")
print("1. Review the generated network.c and network.h files")
print("2. Integrate the models into your STM32 project")
print("3. Configure memory pools based on the .mpool files")
print("4. Test the models on your target STM32 hardware")

# List generated files
print("\nGenerated files:")
for item in output_dir.rglob('*'):
    if item.is_file():
        print(f"  {item.relative_to(output_dir)}")


CONVERSION SUMMARY
CenterFace TFLite → STM32: ✅ Success
MobileFaceNet ONNX → STM32: ✅ Success

Generated files are organized in the ./stm32_output directory

Next steps:
1. Review the generated network.c and network.h files
2. Integrate the models into your STM32 project
3. Configure memory pools based on the .mpool files
4. Test the models on your target STM32 hardware

Generated files:
  centerface_stm32/LICENSE.txt
  centerface_stm32/centerface_OE_3_2_0_Q.json
  centerface_stm32/network_generate_report.txt
  centerface_stm32/centerface_OE_3_3_0.onnx
  centerface_stm32/network_c_info.json
  centerface_stm32/centerface_OE_3_3_0_Q.json
  centerface_stm32/centerface_OE_3_2_0.onnx
  centerface_stm32/network_ecblobs.h
  centerface_stm32/network.h
  centerface_stm32/network.c
  centerface_stm32/network_atonbuf.xSPI2.raw
  mobilefacenet_stm32/LICENSE.txt
  mobilefacenet_stm32/mobilefacenet_real_faces_onnx_OE_3_2_0.onnx
  mobilefacenet_stm32/network_generate_report.txt
  mobilefacenet_stm32